In [0]:
%run ./00_Setup_Environment

In [0]:
df_customer_address = spark.table("bronze.customer_address")

df_address = spark.table("silver.address")
df_customer = spark.table("silver.customer")

In [0]:
df_customer_address.display()

In [0]:
# Agrupar endereços por cliente
df_customer_addresses_grouped = df_customer_address.alias("ca") \
    .join(df_address.alias("a"),
          col("ca.AddressID") == col("a.address_id"), 
          "left") \
    .groupBy("ca.CustomerID") \
    .agg(
        first(when(col("ca.AddressType") == "Main Office", col("full_address")), ignorenulls=True).alias("main_address"),
        first(when(col("ca.AddressType") == "Main Office", col("ca.AddressID")), ignorenulls=True).alias("main_address_id"),
        first(when(col("ca.AddressType") == "Shipping", col("full_address")), ignorenulls=True).alias("shipping_address"),
        first(when(col("ca.AddressType") == "Shipping", col("ca.AddressID")), ignorenulls=True).alias("shipping_address_id"),
        count("ca.AddressID").alias("total_addresses")
    )

df_customer_addresses_grouped.display()

In [0]:
df_silver_customer_address = df_customer_addresses_grouped.alias("ca") \
    .join(df_customer.alias("c"),
          col("ca.CustomerID") == col("c.customer_id"), 
          "left")\
    .select(
        col("c.customer_id"),
        col("ca.main_address_id"),
        col("ca.shipping_address_id"),
        col("full_name"),
        col("c.email_address").alias("email_address"),
        col("c.phone"),
        col("ca.main_address"),
        col("ca.shipping_address"),
        col("ca.total_addresses"),
        col("c.source_modified_date").cast("timestamp"),
        col("c.processed_timestamp")
    )


In [0]:
df_silver_customer_address.display()


In [0]:
path = f"{silver_path}/customer_address"
df_silver_customer_address.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save(path)

spark.sql("""USE adventureworks.silver""")
df_silver_customer_address.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("customer_address")

count = df_silver_customer_address.count()
log_etl("customer", "silver", "SUCCESS", count)

print(f" Silver CustomerAddress: {count} registros")